# 🧹 SAP Report Cleaner (Colab Version)

**Dieses Tool bereinigt SAP-Reports und bereitet sie für die Datenanalyse vor.**

---

## Anleitung:

1. **Zelle 1 ausführen**: Klicken Sie auf ▶️ links neben der Zelle (oder Shift+Enter)
2. **Google Drive verbinden**: Erlauben Sie den Zugriff wenn gefragt
3. **Quelldatei wählen**: SAP-Report hochladen oder aus Drive wählen
4. **Format wählen**: Excel oder CSV
5. **Speicherort wählen**: Ordner in Google Drive angeben
6. **Fertig!** Die bereinigte Datei ist in Ihrem Drive

---


In [ ]:
#@title 🚀 **Ausführen - Klicken Sie auf ▶️ links** { display-mode: "form" }
#@markdown ---
#@markdown ### Dieses Script:
#@markdown - Verbindet mit Google Drive
#@markdown - Lässt Sie eine SAP-Datei auswählen
#@markdown - Bereinigt die Daten automatisch
#@markdown - Speichert die Ergebnisse in Ihrem Drive
#@markdown ---

# ============================================================
# IMPORTS UND SETUP
# ============================================================

import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os
import io

# Google Colab spezifische Imports
from google.colab import drive, files
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

print("✅ Module geladen")

# ============================================================
# KONFIGURATION
# ============================================================

EXPECTED_HEADERS = [
    'Material', 'Functional Loc.', 'Equipment', 'Material Description',
    'Work Ctr', 'Withdrawn', 'W/o resrv.', 'Reserved', 'Reserv.ref',
    'Pstng Date', 'Order', 'ID', 'Message', 'ICt', 'Customer'
]

TEXT_COLUMNS = ['Functional Loc.', 'Equipment', 'Material Description',
                'Work Ctr', 'ID', 'ICt', 'Customer']
DATE_COLUMN = 'Pstng Date'
NUMERIC_COLUMNS = ['Material', 'Withdrawn', 'W/o resrv.', 'Reserved',
                   'Reserv.ref', 'Order', 'Message']

# ============================================================
# HILFSFUNKTIONEN
# ============================================================

def clean_number(value):
    """Bereinigt Zahlenwerte aus SAP-Format."""
    if pd.isna(value) or value is None:
        return None
    val_str = str(value).strip()
    if val_str == '' or val_str == '-':
        return None
    val_str = val_str.replace('\xa0', '').replace(' ', '')
    if ',' in val_str and '.' in val_str:
        val_str = val_str.replace('.', '').replace(',', '.')
    elif ',' in val_str:
        parts = val_str.split(',')
        if len(parts) == 2 and len(parts[1]) <= 2:
            val_str = val_str.replace(',', '.')
        else:
            val_str = val_str.replace(',', '')
    elif '.' in val_str:
        parts = val_str.split('.')
        if len(parts) == 2 and len(parts[1]) == 3 and len(parts[0]) >= 1:
            val_str = val_str.replace('.', '')
    try:
        return int(round(float(val_str)))
    except ValueError:
        return None

def convert_date(value):
    """Konvertiert Datum aus SAP-Format."""
    if pd.isna(value) or value is None:
        return ''
    val_str = str(value).strip()
    if val_str == '':
        return ''
    for fmt in ['%d.%m.%y', '%d.%m.%Y', '%Y-%m-%d']:
        try:
            return datetime.strptime(val_str, fmt).strftime('%d.%m.%Y')
        except ValueError:
            continue
    return val_str

def process_sap_report(content):
    """Verarbeitet SAP-Report Inhalt."""
    lines = content.split('\n')
    all_rows = [line.split('\t') for line in lines]

    # Header finden
    header_row_idx, header_start_col = None, None
    for idx, row in enumerate(all_rows):
        for col_idx, cell in enumerate(row):
            if str(cell).strip().lower() == 'material':
                header_row_idx, header_start_col = idx, col_idx
                break
        if header_row_idx is not None:
            break

    if header_row_idx is None:
        header_row_idx, header_start_col = 3, 2

    # Daten verarbeiten
    cleaned_data, deleted_rows = [], []
    stats = {'total': 0, 'sum_rows': 0, 'empty': 0, 'no_material': 0, 'kept': 0}

    for row_idx in range(header_row_idx + 1, len(all_rows)):
        row = all_rows[row_idx]
        stats['total'] += 1

        if all(str(cell).strip() == '' for cell in row):
            stats['empty'] += 1
            continue

        col_b = str(row[1]).strip() if len(row) > 1 else ''
        if col_b in ['*', '**']:
            stats['sum_rows'] += 1
            deleted_rows.append({'Grund': 'Summenzeile', 'Zeile': row_idx + 1,
                                 'Daten': '\t'.join(str(c) for c in row)})
            continue

        data_row = [str(row[i]).strip() if i < len(row) else ''
                    for i in range(header_start_col, header_start_col + 15)]

        if not data_row[0]:
            stats['no_material'] += 1
            deleted_rows.append({'Grund': 'Keine Materialnummer', 'Zeile': row_idx + 1,
                                 'Daten': '\t'.join(data_row)})
            continue

        cleaned_data.append(data_row)
        stats['kept'] += 1

    # DataFrame erstellen
    df = pd.DataFrame(cleaned_data, columns=EXPECTED_HEADERS)
    df_deleted = pd.DataFrame(deleted_rows)

    # Datentypen konvertieren
    for col in NUMERIC_COLUMNS:
        if col in df.columns:
            df[col] = df[col].apply(clean_number)
    if DATE_COLUMN in df.columns:
        df[DATE_COLUMN] = df[DATE_COLUMN].apply(convert_date)

    return df, df_deleted, stats

print("✅ Funktionen geladen")

# ============================================================
# GOOGLE DRIVE VERBINDEN
# ============================================================

print("\n📁 Verbinde mit Google Drive...")
drive.mount('/content/drive')
print("✅ Google Drive verbunden!")

# ============================================================
# INTERAKTIVE OBERFLÄCHE
# ============================================================

# Globale Variablen
result_df = None
result_deleted = None
result_stats = None
source_filename = None

# UI Elemente
output_area = widgets.Output()

source_dropdown = widgets.Dropdown(
    options=['📤 Vom Computer hochladen', '📁 Aus Google Drive wählen'],
    value='📤 Vom Computer hochladen',
    description='Quelle:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='350px')
)

drive_path_input = widgets.Text(
    value='/content/drive/MyDrive/',
    description='Drive-Pfad:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px'),
    placeholder='z.B. /content/drive/MyDrive/Downloads/report.txt'
)

format_dropdown = widgets.Dropdown(
    options=['📊 Excel (.xlsx) - Mit gelöschten Zeilen', '📄 CSV (.csv) - Nur bereinigte Daten'],
    value='📊 Excel (.xlsx) - Mit gelöschten Zeilen',
    description='Format:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

# Speicherort-Auswahl
save_location = widgets.Dropdown(
    options=['💾 Auf meinen Computer herunterladen', '📁 In Google Drive speichern'],
    value='💾 Auf meinen Computer herunterladen',
    description='Speichern:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

save_path_input = widgets.Text(
    value='/content/drive/MyDrive/',
    description='Drive-Pfad:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='500px'),
    placeholder='z.B. /content/drive/MyDrive/Bereinigt/'
)

upload_btn = widgets.Button(description='📤 Datei laden', button_style='primary',
                            layout=widgets.Layout(width='200px'))
process_btn = widgets.Button(description='🚀 Verarbeiten & Speichern', button_style='success',
                             layout=widgets.Layout(width='200px'))
status_label = widgets.HTML(value='<b>Status:</b> Bereit')

def on_upload_click(b):
    global result_df, result_deleted, result_stats, source_filename
    with output_area:
        clear_output()
        if '📤' in source_dropdown.value:
            print("📤 Bitte Datei auswählen...")
            uploaded = files.upload()
            if not uploaded:
                print("❌ Keine Datei ausgewählt")
                return
            source_filename = list(uploaded.keys())[0]
            content = uploaded[source_filename].decode('utf-8', errors='replace')
        else:
            file_path = drive_path_input.value
            if not os.path.exists(file_path):
                print(f"❌ Datei nicht gefunden: {file_path}")
                return
            source_filename = os.path.basename(file_path)
            with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                content = f.read()

        print(f"📄 Datei geladen: {source_filename}")
        print("⏳ Verarbeite...")
        result_df, result_deleted, result_stats = process_sap_report(content)
        print(f"\n✅ Verarbeitung abgeschlossen!")
        print(f"\n📊 Statistik:")
        print(f"   Bereinigte Zeilen: {result_stats['kept']}")
        print(f"   Summenzeilen entfernt: {result_stats['sum_rows']}")
        print(f"   Ohne Materialnr. entfernt: {result_stats['no_material']}")
        print(f"\n📋 Vorschau (erste 5 Zeilen):")
        display(result_df.head())
        status_label.value = f'<b>Status:</b> ✅ {result_stats["kept"]} Zeilen bereit'

def on_process_click(b):
    global result_df, result_deleted, source_filename
    with output_area:
        if result_df is None:
            print("❌ Bitte zuerst eine Datei laden!")
            return
        base_name = Path(source_filename).stem
        
        # Prüfe ob Download auf Computer oder Google Drive
        download_to_pc = '💾' in save_location.value
        
        if download_to_pc:
            # Direkt auf Computer herunterladen
            if '📊' in format_dropdown.value:
                output_filename = f"{base_name}_cleaned.xlsx"
                output_path = f"/content/{output_filename}"
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    result_df.to_excel(writer, sheet_name='Bereinigte Daten', index=False)
                    if not result_deleted.empty:
                        result_deleted.to_excel(writer, sheet_name='Gelöschte Zeilen', index=False)
                print(f"\n💾 Excel erstellt: {output_filename}")
            else:
                output_filename = f"{base_name}_cleaned.csv"
                output_path = f"/content/{output_filename}"
                result_df.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
                print(f"\n💾 CSV erstellt: {output_filename}")
            
            print("📥 Starte Download...")
            files.download(output_path)
            print(f"\n✅ Fertig! Die Datei wird in Ihren Downloads-Ordner heruntergeladen.")
            status_label.value = f'<b>Status:</b> ✅ Download gestartet!'
        else:
            # In Google Drive speichern
            save_dir = save_path_input.value.rstrip('/')
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
                print(f"📁 Ordner erstellt: {save_dir}")

            if '📊' in format_dropdown.value:
                output_path = f"{save_dir}/{base_name}_cleaned.xlsx"
                with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                    result_df.to_excel(writer, sheet_name='Bereinigte Daten', index=False)
                    if not result_deleted.empty:
                        result_deleted.to_excel(writer, sheet_name='Gelöschte Zeilen', index=False)
                print(f"\n💾 Excel gespeichert: {output_path}")
            else:
                output_path = f"{save_dir}/{base_name}_cleaned.csv"
                result_df.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
                print(f"\n💾 CSV gespeichert: {output_path}")

            print(f"\n✅ Fertig! Die Datei ist in Ihrem Google Drive.")
            status_label.value = f'<b>Status:</b> ✅ Gespeichert!'

upload_btn.on_click(on_upload_click)
process_btn.on_click(on_process_click)

# Layout anzeigen
print("\n" + "="*60)
print("  📋 SAP REPORT CLEANER")
print("="*60)
display(widgets.HTML('<h3>1️⃣ Quelldatei</h3>'))
display(source_dropdown)
display(widgets.HTML('<p><i>Für Drive: Pfad eingeben, z.B. /content/drive/MyDrive/Downloads/report.txt</i></p>'))
display(drive_path_input)
display(upload_btn)
display(widgets.HTML('<h3>2️⃣ Ausgabeformat</h3>'))
display(format_dropdown)
display(widgets.HTML('<h3>3️⃣ Speicherort</h3>'))
display(save_location)
display(widgets.HTML('<p><i>Bei Google Drive - Ordner angeben:</i></p>'))
display(save_path_input)
display(widgets.HTML('<h3>4️⃣ Verarbeiten</h3>'))
display(process_btn)
display(status_label)
display(widgets.HTML('<hr><h3>📊 Ergebnis:</h3>'))
display(output_area)


---

## 💡 Tipps

### 📂 Datei auswählen (Upload)
- Wählen Sie **"Vom Computer hochladen"** → Es öffnet sich ein **Datei-Dialog**
- Navigieren Sie zu Ihrem **Downloads-Ordner** und wählen Sie die SAP-Datei
- **Kein Pfad eingeben nötig!**

### 💾 Datei speichern
| Option | Was passiert |
|--------|--------------|
| **Auf Computer herunterladen** | Öffnet Browser-Download → Datei landet in Downloads |
| **In Google Drive speichern** | Speichert direkt in Ihrem Drive-Ordner |

### Excel vs CSV:
| Format | Vorteile |
|--------|----------|
| **Excel** | 2 Tabellenblätter (Daten + Gelöschte), direkt in Excel öffnen |
| **CSV** | Kleiner, universell kompatibel |

---

## ❓ Problemlösung

| Problem | Lösung |
|---------|--------|
| Download startet nicht | Popup-Blocker für colab.google.com deaktivieren |
| "Datei nicht gefunden" | Bei Drive: Pfad prüfen |
| Keine Daten | Sicherstellen dass Datei Tab-getrennt ist (.txt) |

---

*SAP Report Cleaner v1.1 - Colab Version - Januar 2026*
